# GERADOR DE POEMAS COM IA E FEEDBACK

## Objetivo do Sistema

Este projeto implementa um **sistema completo de geração de poemas** usando IA generativa via API, com as seguintes funcionalidades:

- **Geração de poemas**: Uso de LLM (Large Language Model) via API
- **Feedback do usuário**: Coleta estruturada de avaliação do poema
- **Novo Chat vs Chat Contínuo**: Diferenciação entre iniciar nova conversa ou continuar com histórico
- **Interface interativa**: Widgets no Jupyter para experiência intuitiva
- **Melhoria automática**: Refinamento de prompts baseado no feedback

## Responsabilidades

- **Grupo 1**: Integração com LLM e chamadas à API
- **Grupo 2**: Interface interativa com widgets
- **Grupo 3**: Sistema de feedback e refinamento de prompts

---

In [ ]:
!pip install gradio==5.40.0
!pip install -U -q "google-genai"

In [ ]:
# Importações
import gradio as gr
from google import genai

# --- Configuração da API do Gemini ---
try:
    client = genai.Client()
    model_name = "gemini-2.5-pro"
except Exception as e:
    print(f"Erro ao configurar a API do Gemini: {e}")
    print("Certifique-se de ter adicionado sua GEMINI_API_KEY como um Secret no Kaggle.")
    client = None
    model_name = None


# --- Função de Backend ---
def gerar_poema(tema, estilo, tamanho, temperatura, poema_atual, refinamentos_list, chat_atual):
    """
    Gera ou refina um poema com base nos parâmetros.
    Mantém o mesmo chat para refinamentos ou cria novo para poemas novos.
    """
    temp_normalizada = temperatura / 100.0
    system_instruction = "Você é um poeta de renome, especialista em vários estilos. Sua única saída deve ser o poema (refinado ou gerado) em português, sem explicações, saudações ou markdown adicional."

    prompt = ""
    criar_novo_chat = False

    # Se há refinamentos a aplicar (usa o chat existente)
    if refinamentos_list and len(refinamentos_list) > 0:
        prompt = f"Poema atual:\n{poema_atual}\n\nAplique as seguintes modificações:\n"
        for i, ref in enumerate(refinamentos_list, 1):
            prompt += f"{i}. {ref['acao']}: {ref['alvo']}"
            if ref['novo_conteudo']:
                prompt += f" - Novo conteúdo: {ref['novo_conteudo']}"
            prompt += "\n"
        criar_novo_chat = False
    else:
        # Geração inicial (cria novo chat)
        prompt = f"Crie um poema {estilo} sobre: {tema}."
        if tamanho == "Curto (1-2 estrofes)":
            prompt += " O poema deve ter 1 ou 2 estrofes."
        elif tamanho == "Médio (3-4 estrofes)":
            prompt += " O poema deve ter 3 ou 4 estrofes."
        else:
            prompt += " O poema deve ter 5 ou mais estrofes."
        criar_novo_chat = True

    if tema == "":
        return f"Defina um tema", poema_atual, gr.Tabs(selected=1), chat_atual

    if client and model_name:
        try:
            # Criar novo chat ou usar o existente
            if criar_novo_chat or chat_atual is None:
                chat = client.chats.create(model=model_name, config={
                    "temperature": temp_normalizada,
                    "system_instruction": system_instruction
                })
            else:
                chat = chat_atual

            response = chat.send_message(prompt)

            if response.text:
                return response.text, response.text, gr.Tabs(selected=1), chat
            else:
                return "Não foi possível gerar/refinar. Tente novamente.", poema_atual, gr.Tabs(selected=1), chat
        except Exception as e:
            error_message = f"Erro na API do Gemini: {e}"
            print(error_message)
            return f"Ocorreu um erro: {error_message}", poema_atual, gr.Tabs(selected=1), chat_atual

    return "API não configurada.", poema_atual, gr.Tabs(selected=1), None


# --- CSS Personalizado ---
custom_css = """
.gradio-container {
    max-width: 900px !important;
    margin: auto !important;
}

h1 {
    text-align: center;
    color: #2c3e50;
    font-size: 2.5em;
    margin-bottom: 10px;
}

.subtitle {
    text-align: center;
    color: #7f8c8d;
    font-size: 1.2em;
    margin-bottom: 30px;
}

.generate-btn {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    border: none !important;
    color: white !important;
    font-size: 1.1em !important;
    padding: 15px 30px !important;
    border-radius: 10px !important;
    font-weight: bold !important;
    transition: transform 0.2s !important;
}

.generate-btn:hover {
    transform: scale(1.05) !important;
}

.exemplo-btn {
    background: #3498db !important;
    color: white !important;
    border-radius: 8px !important;
}

.limpar-btn {
    background: #95a5a6 !important;
    color: white !important;
    border-radius: 8px !important;
}

.button-row {
    display: flex !important;
    justify-content: space-between !important;
    gap: 10px !important;
}

.refinamento-box {
    border: 2px solid #667eea;
    border-radius: 10px;
    padding: 15px;
    margin: 10px 0;
    background: #f8f9fa;
}
"""

# --- Interface Gradio ---
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="Gerador de Poemas IA") as demo:
    poema_state = gr.State(value="")
    refinamentos_state = gr.State(value=[])  # Lista de refinamentos
    chat_state = gr.State(value=None)  # Armazena o objeto chat

    gr.Markdown("# Gerador de Poemas com IA")
    gr.Markdown("<p class='subtitle'>✨ Transforme suas ideias em versos únicos e inspiradores</p>")

    with gr.Tabs() as tabs:
        # ABA 1: Definição do Tema
        with gr.Tab("📝 Criar Poema", id=0):
            gr.Markdown("### Defina o tema e estilo do seu poema")

            with gr.Row():
                with gr.Column(scale=2):
                    tema_input = gr.Textbox(
                        label="Tema do Poema",
                        placeholder="Ex: A beleza do pôr do sol, saudade, amor impossível...",
                        lines=3,
                        max_lines=5
                    )

                    estilo_input = gr.Dropdown(
                        label="Estilo do Poema",
                        choices=[
                            "Livre (sem estrutura específica)",
                            "Romântico",
                            "Haicai (estilo japonês)",
                            "Soneto",
                            "Moderno",
                            "Clássico",
                            "Melancólico",
                            "Alegre e descontraído"
                        ],
                        value="Livre (sem estrutura específica)"
                    )

                    tamanho_input = gr.Radio(
                        label="Tamanho do Poema",
                        choices=["Curto (1-2 estrofes)", "Médio (3-4 estrofes)", "Longo (5+ estrofes)"],
                        value="Médio (3-4 estrofes)"
                    )

                with gr.Column(scale=1):
                    gr.Markdown("### ⚙️ Configurações Avançadas")

                    temperatura_input = gr.Slider(
                        minimum=0,
                        maximum=100,
                        value=70,
                        step=1,
                        label="Criatividade",
                        info="Maior = mais criativo e variado"
                    )

                    gr.Markdown("---")
                    gr.Markdown("### 💡 Exemplos de Temas")
                    gr.Markdown("""
                    - O som da chuva à noite
                    - Um gato observando a lua
                    - A solidão na cidade grande
                    - Memórias de infância
                    """)

            with gr.Row(elem_classes=["button-row"]):
                gerar_btn = gr.Button("🚀 Gerar Poema", variant="primary", elem_classes=["generate-btn"])
                limpar_btn = gr.Button("🗑️ Limpar", variant="secondary", elem_classes=["limpar-btn", "generate-btn"])

        # ABA 2: Resultado do Poema
        with gr.Tab("📜 Seu Poema", id=1):
            gr.Markdown("### ✨ Seu poema foi gerado!")

            poema_output = gr.Textbox(
                label="",
                lines=15,
                max_lines=20,
                show_label=False,
                interactive=True,
                placeholder="Seu poema aparecerá aqui após a geração..."
            )

            gr.Markdown("---")

            # Botão para mostrar/ocultar refinamentos
            with gr.Row():
                mostrar_refinamentos_btn = gr.Button("➕ Adicionar Refinamentos", variant="secondary")

            # Container de refinamentos (inicialmente oculto)
            refinamentos_container = gr.Column(visible=False)

            with refinamentos_container:
                gr.Markdown("### ✂️ Refinamentos")

                # Container dinâmico para múltiplos blocos
                refinamentos_accordion = gr.Accordion("Refinamentos Adicionados", open=True)

                with refinamentos_accordion:
                    refinamentos_display = gr.Markdown("*Nenhum refinamento adicionado ainda*")

                gr.Markdown("#### Adicionar Novo Refinamento")

                with gr.Row(elem_classes=["refinamento-box"]):
                    with gr.Column():
                        acao_temp = gr.Radio(
                            label="Ação",
                            choices=["Adicionar", "Editar", "Remover"],
                            value="Editar"
                        )
                        alvo_temp = gr.Textbox(
                            label="Trecho/Alvo",
                            placeholder="Ex: 'A terceira estrofe', 'A palavra vento'",
                            lines=1
                        )
                        novo_conteudo_temp = gr.Textbox(
                            label="Novo Conteúdo",
                            placeholder="Digite o novo texto",
                            lines=2
                        )

                with gr.Row():
                    adicionar_refinamento_btn = gr.Button("➕ Adicionar à Lista", variant="secondary")
                    limpar_refinamentos_btn = gr.Button("🗑️ Limpar Todos", variant="secondary")

                refinar_btn = gr.Button("🔄 Aplicar Refinamentos", variant="primary", elem_classes=["generate-btn"])

            with gr.Row():
                copiar_btn = gr.Button("📋 Copiar Poema", variant="secondary")
                novo_btn = gr.Button("✏️ Criar Novo Poema", variant="primary")


    # --- Funções de Controle ---
    def toggle_refinamentos():
        return gr.Column(visible=True)


    def adicionar_refinamento(refinamentos, acao, alvo, novo_conteudo):
        if not alvo:
            return refinamentos, formatar_refinamentos(refinamentos), "", "", ""

        refinamentos.append({
            'acao': acao,
            'alvo': alvo,
            'novo_conteudo': novo_conteudo
        })

        return refinamentos, formatar_refinamentos(refinamentos), "Editar", "", ""


    def limpar_refinamentos():
        return [], "*Nenhum refinamento adicionado ainda*"


    def formatar_refinamentos(refinamentos):
        if not refinamentos:
            return "*Nenhum refinamento adicionado ainda*"

        texto = ""
        for i, ref in enumerate(refinamentos, 1):
            texto += f"**{i}. {ref['acao']}**: {ref['alvo']}"
            if ref['novo_conteudo']:
                texto += f" → *{ref['novo_conteudo']}*"
            texto += "\n\n"
        return texto


    def limpar_campos():
        return "", "Livre (sem estrutura específica)", "Médio (3-4 estrofes)", 70, "", [], None


    def voltar_criar():
        return gr.Tabs(selected=0)


    # --- Conectar Eventos ---
    # Geração inicial (cria novo chat)
    gerar_btn.click(
        fn=lambda t, e, tam, temp, p, r, c: gerar_poema(t, e, tam, temp, p, [], c),
        inputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state, refinamentos_state,
                chat_state],
        outputs=[poema_output, poema_state, tabs, chat_state]
    )

    # Toggle refinamentos
    mostrar_refinamentos_btn.click(
        fn=toggle_refinamentos,
        outputs=[refinamentos_container]
    )

    # Adicionar refinamento à lista
    adicionar_refinamento_btn.click(
        fn=adicionar_refinamento,
        inputs=[refinamentos_state, acao_temp, alvo_temp, novo_conteudo_temp],
        outputs=[refinamentos_state, refinamentos_display, acao_temp, alvo_temp, novo_conteudo_temp]
    )

    # Limpar refinamentos
    limpar_refinamentos_btn.click(
        fn=limpar_refinamentos,
        outputs=[refinamentos_state, refinamentos_display]
    )

    # Aplicar refinamentos (usa o chat existente)
    refinar_btn.click(
        fn=gerar_poema,
        inputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state, refinamentos_state,
                chat_state],
        outputs=[poema_output, poema_state, tabs, chat_state]
    ).then(
        fn=limpar_refinamentos,
        outputs=[refinamentos_state, refinamentos_display]
    )

    # Atualizar state quando editar manualmente
    poema_output.change(fn=lambda x: x, inputs=poema_output, outputs=poema_state)

    # Outros botões
    limpar_btn.click(fn=limpar_campos, outputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state,
                                                refinamentos_state, chat_state])
    novo_btn.click(fn=voltar_criar, outputs=tabs)

# --- Lançar Interface ---
demo.launch(inline=True, share=True)